# Simulation template

In this notebook we run the machine learning analysis of topological phase transitions occurring  in both nearest-neighbours SSH models (ssh1) and second neighbours models (ssh2) as decribed in the paper [Machine learning topological phases in real space](https://arxiv.org/abs/1901.01963).

## Defining parameters

In [1]:
%%time
%load_ext autoreload
%autoreload 2
from simulation import *

CPU times: user 961 ms, sys: 381 ms, total: 1.34 s
Wall time: 1.81 s


In [2]:
%%time
### Dataset and simulation parameters
csv_path = SSH1_PERIODIC_100_6561_CSV 
model_name = "DecisionTreeClassifier"
model_kw = {"criterion":"entropy"}
allowed_windings = [0,1]
simulation_dir = SSH1_PERIODIC_100_6561_SIMULATION_DIR
val_split = 0.9  
features_to_use = None 
shuffle_features = False
random_state = 137                    

### Running a simulation
n_experiments = 5
start_n = 0
fit_params = None
shuffle_rows = True
pred_params = None
random_features = False
######### DON'T SET THIS TO TRUE UNLESS YOUR DATASET IS SMALL!! WILL FLOOD YOUR MEMORY!!!
store_in_lists = False   
########## BELOW ARE THE PARAMETERS THAT CONTROL WHAT WILL BE SAVED
save_eigenvector=True
save_hamiltonian=True 
save_accuracy=True 
save_models=True

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.01 µs


In [3]:
# Parameters
model_kw = {"criterion": "entropy", "n_estimators": 25, "n_jobs": -1}
allowed_windings = [-1, 0, 1, 2]
val_split = 0.5
features_to_use = None
shuffle_features = False
n_experiments = 100
start_n = 0
fit_params = None
shuffle_rows = True
pred_params = None
random_features = False
store_in_lists = False
save_eigenvector = True
save_hamiltonian = True
save_accuracy = True
save_models = True
csv_path = "/home/rio/ssh_csvs/ssh2/periodic_220_6561.csv"
model_name = "RandomForestClassifier"
simulation_dir = "/home/rio/ssh_simulations/ssh2/periodic_220_6561"
random_state = 401


In [4]:
%%time
#Starting an instance of Simulation with a decision tree
model = DecisionTreeClassifier(criterion="entropy")
#dict_args = {"csv_path":csv_path, "model":model, "allowed_windings":allowed_windings,\
#             "simulation_dir":simulation_dir, "val_split":val_split, "features_to_use":features_to_use,\
#            "shuffle_features":shuffle_features, "random_state":random_state}
simulation = Simulation(csv_path,model_name,model_kw,allowed_windings,simulation_dir,val_split,features_to_use,\
                       shuffle_features,random_state)
#simulation = Simulation(**dict_args)

print("Info on all data: \n")
simulation.dataframe.info()
simulation.dataframe.head()

/home/rio/ml_top_phases/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Info on all data: 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1443420 entries, 0 to 1443419
Columns: 228 entries, id to feat219
dtypes: float64(223), int32(3), object(2)
memory usage: 2.4+ GB
CPU times: user 30.4 s, sys: 3.51 s, total: 33.9 s
Wall time: 42.1 s


,id,path,t1,t2,winding,phase,pred_phase,type_of,feat0,feat1,...,feat210,feat211,feat212,feat213,feat214,feat215,feat216,feat217,feat218,feat219
0,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,0.130846,-0.300844,...,0.080274,0.115552,0.174807,-0.244872,-0.319478,0.059064,0.000568,0.244891,0.300844,-0.130846
1,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,-0.130846,-0.300844,...,-0.080274,0.115552,-0.174807,-0.244872,0.319478,0.059064,-0.000568,0.244891,-0.300844,-0.130846
2,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,-0.139243,-0.040832,...,0.177651,-0.085976,-0.069808,-0.109102,-0.140759,0.184935,0.184071,-0.052390,-0.040832,-0.139243
3,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,0.048483,0.015611,...,-0.065026,0.044350,0.035714,0.029502,0.046097,-0.070102,-0.070512,0.025241,0.015611,0.048483
4,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,-0.005059,0.009352,...,-0.013734,0.009909,0.002329,0.008250,0.011761,-0.009332,-0.002720,-0.004584,-0.009352,0.005059


#### Checking initialization

In [5]:
%%time
n_features = simulation.n_features
n_hamiltonians = simulation.n_hamiltonians
n_ts = simulation.n_ts

print("n_features: ", n_features)
print("n_hamiltonians: ", n_hamiltonians)
print("n_ts: ", n_ts)

n_features:  220
n_hamiltonians:  6561
n_ts:  2
CPU times: user 615 µs, sys: 0 ns, total: 615 µs
Wall time: 358 µs


In [6]:
%%time
n_total = len(simulation.dataframe)
n_train = len(simulation.dataframe[simulation.dataframe.type_of == "train"])
n_val = len(simulation.dataframe[simulation.dataframe.type_of == "val"])
n_test = len(simulation.dataframe[simulation.dataframe.type_of == "test"])
n_train_hamiltonians = len(simulation.train_ids)
n_val_hamiltonians = len(simulation.val_ids)
n_test_hamiltonians = len(simulation.test_ids)
n_total_hamiltonians = n_train_hamiltonians + n_val_hamiltonians + n_test_hamiltonians

print("% train: ", n_train/n_total)
print("% val: ",  n_val/n_total)
print("% test: ", n_test/n_total)
print("% train + val + test: ", (n_train+n_val+n_test)/n_total)
print("\n")
print("number of train hamiltonians: ", n_train_hamiltonians)
print("number of val hamiltonians: ", n_val_hamiltonians)
print("number of test hamiltonians: ", n_test_hamiltonians)
print("total number of hamiltonians: ", n_total_hamiltonians)
print("\n")
print("train ids: ", simulation.train_ids)
print("val ids: ", simulation.val_ids)
print("test ids: ", simulation.test_ids)

% train:  0.8414875781130925
% val:  0.0
% test:  0.15851242188690748
% train + val + test:  1.0


number of train hamiltonians:  5521
number of val hamiltonians:  0
number of test hamiltonians:  1040
total number of hamiltonians:  6561


train ids:  [13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 201, 202, 203, 204, 205, 206, 207, 

## Running a simulation

In [7]:
%%time
simulation.run_simulation(n_experiments, start_n, fit_params, shuffle_rows, pred_params, random_features, \
                          store_in_lists, save_eigenvector, save_hamiltonian, save_accuracy,\
                          save_models)

simulation.dataframe.head(10)

running experiments:   0%|          | 0/100 [00:00<?, ?it/s]

THIS IS  predict_params:  None
THIS IS  the type of predict_params:  <class 'NoneType'>


running experiments:   1%|          | 1/100 [02:23<3:57:32, 143.97s/it]

THIS IS  predict_params:  None
THIS IS  the type of predict_params:  <class 'NoneType'>


running experiments:   2%|▏         | 2/100 [04:48<3:55:35, 144.24s/it]

...

THIS IS  predict_params:  None
THIS IS  the type of predict_params:  <class 'NoneType'>


running experiments: 100%|██████████| 100/100 [3:59:17<00:00, 141.30s/it]

running experiments: 100%|██████████| 100/100 [3:59:17<00:00, 143.58s/it]

CPU times: user 1d 2h 41min 32s, sys: 53.7 s, total: 1d 2h 42min 26s
Wall time: 3h 59min 17s


,id,path,t1,t2,winding,phase,pred_phase,type_of,feat0,feat1,...,feat210,feat211,feat212,feat213,feat214,feat215,feat216,feat217,feat218,feat219
0,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,-1,test,0.130846,-0.300844,...,0.080274,0.115552,0.174807,-0.244872,-0.319478,0.059064,0.000568,0.244891,0.300844,-0.130846
1,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,-1,test,-0.130846,-0.300844,...,-0.080274,0.115552,-0.174807,-0.244872,0.319478,0.059064,-0.000568,0.244891,-0.300844,-0.130846
2,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,-1,test,-0.139243,-0.040832,...,0.177651,-0.085976,-0.069808,-0.109102,-0.140759,0.184935,0.184071,-0.052390,-0.040832,-0.139243
3,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,0.048483,0.015611,...,-0.065026,0.044350,0.035714,0.029502,0.046097,-0.070102,-0.070512,0.025241,0.015611,0.048483
4,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,-1,test,-0.005059,0.009352,...,-0.013734,0.009909,0.002329,0.008250,0.011761,-0.009332,-0.002720,-0.004584,-0.009352,0.005059
5,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,-0.050074,-0.011117,...,0.049582,0.005255,0.001717,-0.054642,-0.055678,0.050847,0.051142,-0.002840,-0.011117,-0.050074
6,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,-1,test,0.009938,-0.026818,...,-0.013334,0.022656,0.023129,-0.013172,-0.023569,-0.006879,-0.004162,0.025408,0.026818,-0.009938
7,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,-1,test,-0.014007,0.025469,...,-0.039242,0.030036,0.008341,0.021972,0.032295,-0.027020,-0.008046,-0.011692,-0.025469,0.014007
8,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,-1,test,-0.053983,-0.019696,...,0.075379,-0.071666,-0.058670,-0.017526,-0.045907,0.086976,0.089522,-0.040115,-0.019696,-0.053983
9,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,0.068700,0.013277,...,-0.056877,-0.023603,-0.014327,0.078240,0.077328,-0.060835,-0.062904,-0.003322,0.013277,0.068700


#### Viewing a random sample

In [8]:
%%time
simulation.dataframe.sample(frac=0.1, replace=False)

CPU times: user 259 ms, sys: 0 ns, total: 259 ms
Wall time: 259 ms


,id,path,t1,t2,winding,phase,pred_phase,type_of,feat0,feat1,...,feat210,feat211,feat212,feat213,feat214,feat215,feat216,feat217,feat218,feat219
434656,1975,H_-0.8_-0.45_-0.00836797.dat,-0.80,-0.45,-0.008368,0,0,val,-0.061663,0.068614,...,0.082597,-0.094011,0.022668,-0.073907,-0.028952,-0.009051,-0.067927,0.043380,-0.068614,0.061663
1330469,6047,H_1.7_0.65_-1.00207.dat,1.70,0.65,-1.002066,-1,-1,val,-0.023580,-0.000248,...,-0.038742,-0.000485,0.008082,-0.010292,-0.007022,0.008964,0.019802,0.016546,-0.000248,-0.023580
307651,1398,H_-1.15_-0.95_-1.02198.dat,-1.15,-0.95,-1.021982,-1,-1,test,-0.000160,0.067798,...,-0.095038,-0.007566,0.056939,-0.056138,0.007932,0.099201,-0.060810,-0.081506,0.067798,-0.000160
556922,2531,H_-0.45_-1_1.0068.dat,-0.45,-1.00,1.006804,1,1,val,-0.068489,-0.002885,...,0.003824,-0.010321,0.044694,-0.013810,-0.126689,0.039813,-0.030962,0.111347,-0.002885,-0.068489
1330546,6047,H_1.7_0.65_-1.00207.dat,1.70,0.65,-1.002066,-1,-1,val,-0.112745,-0.001853,...,-0.129938,-0.024863,0.077884,0.023528,-0.001579,-0.042642,0.038654,-0.067305,0.001853,0.112745
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387185,1759,H_-0.95_0.9_-8.79368e-05.dat,-0.95,0.90,-0.000088,0,0,train,-0.053405,-0.039623,...,-0.059977,-0.112795,0.014920,-0.065708,-0.018820,0.018390,-0.098877,-0.033984,-0.039623,-0.053405
741503,3370,H_0.05_0.45_-0.000135566.dat,0.05,0.45,-0.000136,0,0,val,-0.126662,-0.059678,...,-0.014037,0.071195,-0.092580,-0.054318,-0.006211,-0.082677,0.102617,0.047739,0.059678,0.126662
346039,1572,H_-1.05_-0.35_-0.00426106.dat,-1.05,-0.35,-0.004261,0,0,train,-0.039874,-0.015729,...,0.049226,-0.035296,-0.091115,0.051914,0.028842,-0.003383,-0.078198,0.044141,0.015729,0.039874
31276,142,H_-1.95_1.05_-0.000307298.dat,-1.95,1.05,-0.000307,0,0,train,-0.030685,0.017138,...,0.025756,0.062831,0.015320,0.065107,-0.001978,0.037990,-0.003107,0.036335,-0.017138,0.030685


#### Checking train/val/test splits again

In [9]:
%%time
n_total = len(simulation.dataframe)
n_train = len(simulation.dataframe[simulation.dataframe.type_of == "train"])
n_val = len(simulation.dataframe[simulation.dataframe.type_of == "val"])
n_test = len(simulation.dataframe[simulation.dataframe.type_of == "test"])
n_train_hamiltonians = len(simulation.train_ids)
n_val_hamiltonians = len(simulation.val_ids)
n_test_hamiltonians = len(simulation.test_ids)
n_total_hamiltonians = n_train_hamiltonians + n_val_hamiltonians + n_test_hamiltonians

print("% train: ", n_train/n_total)
print("% val: ",  n_val/n_total)
print("% test: ", n_test/n_total)
print("% train + val + test: ", (n_train+n_val+n_test)/n_total)
print("\n")
print("number of train hamiltonians: ", n_train_hamiltonians)
print("number of val hamiltonians: ", n_val_hamiltonians)
print("number of test hamiltonians: ", n_test_hamiltonians)
print("total number of hamiltonians: ", n_total_hamiltonians)
print("\n")
print("train ids: ", simulation.train_ids)
print("val ids: ", simulation.val_ids)
print("test ids: ", simulation.test_ids)

% train:  0.4208199969516842
% val:  0.42066758116140834
% test:  0.15851242188690748
% train + val + test:  1.0


number of train hamiltonians:  2761
number of val hamiltonians:  2760
number of test hamiltonians:  1040
total number of hamiltonians:  6561


train ids:  [4612, 1777, 4949, 2591, 2638, 2395, 2331, 5548, 234, 1423, 6304, 5589, 2263, 4548, 3844, 5312, 2807, 807, 2578, 2581, 1086, 3920, 5033, 4542, 3761, 2763, 3025, 961, 224, 5618, 485, 4984, 5683, 5873, 4892, 4382, 5071, 3782, 444, 388, 5567, 5369, 3295, 1187, 3845, 2114, 3395, 3316, 2689, 5852, 6520, 4113, 761, 402, 5781, 2475, 3038, 2088, 5010, 3307, 2617, 5241, 2176, 6522, 4743, 3487, 6498, 5137, 3499, 3465, 5178, 3136, 2104, 3615, 5211, 3423, 6215, 4717, 4130, 6214, 3698, 2611, 616, 6019, 3961, 2913, 4521, 1460, 2364, 555, 562, 2630, 670, 669, 2316, 956, 2739, 1087, 4022, 6174, 4072, 5814, 3736, 1194, 706, 3933, 1875, 4720, 197, 144, 1786, 5773, 5457, 587, 2992, 5458, 6141, 2909, 609, 3231, 3600, 2407, 1409, 6297, 4164,

#### Checking summaries

In [10]:
%%time
ham_summary = simulation.hamiltonian_summary
print("length of ham_summary: ", len(ham_summary))
ham_summary

length of ham_summary:  6561
CPU times: user 82 µs, sys: 0 ns, total: 82 µs
Wall time: 58.2 µs


,t1,t2,type_of,-1,0,1,2,phase,pred_phase
id,,,,,,,,,
0,-2.0,-2.00,test,0.359091,0.359091,0.168182,0.113636,999,0
1,-2.0,-1.95,test,0.359091,0.368182,0.186364,0.086364,-1,0
2,-2.0,-1.90,test,0.368182,0.327273,0.209091,0.095455,-1,-1
3,-2.0,-1.85,test,0.377273,0.327273,0.204545,0.090909,-1,-1
4,-2.0,-1.80,test,0.409091,0.322727,0.200000,0.068182,-1,-1
...,...,...,...,...,...,...,...,...,...
6556,2.0,1.80,test,0.563636,0.177273,0.040909,0.218182,-1,-1
6557,2.0,1.85,test,0.490909,0.163636,0.081818,0.263636,-1,-1
6558,2.0,1.90,test,0.386364,0.213636,0.100000,0.300000,-1,-1


In [11]:
eigen_summary = simulation.eigenvector_summary
print("length of ham_summary: ", len(eigen_summary))
eigen_summary

length of ham_summary:  1443420


,id,phase,pred_phase,type_of
0,0,999,-1,test
1,0,999,-1,test
2,0,999,-1,test
3,0,999,1,test
4,0,999,-1,test
...,...,...,...,...
1443415,6560,999,1,test
1443416,6560,999,2,test
1443417,6560,999,1,test
1443418,6560,999,1,test


#### Checking accuracies

In [12]:
simulation.accuracy

{'eigenvector_train': 0.999828784037404,
 'eigenvector_val': 0.9593214756258235,
 'eigenvector_test': 0.6440258228383956,
 'hamiltonian_train': 1.0,
 'hamiltonian_val': 0.9952898550724638,
 'hamiltonian_test': 0.9126891734575088}

#### Checking data stored in  memory

In [13]:
ham_summary_list = simulation.hamiltonian_summary_list
ham_summary_list

[]

In [14]:
eigen_summary_list = simulation.eigenvector_summary_list
eigen_summary_list

[]

In [15]:
accuracy_list = simulation.accuracy_list
accuracy_list

{'eigenvector_train': [],
 'eigenvector_val': [],
 'eigenvector_test': [],
 'hamiltonian_train': [],
 'hamiltonian_val': [],
 'hamiltonian_test': []}